# Preparation

Checks that notebook has access to GPU. If not go to `Runtime` -> `change runtime type`.



In [ ]:
!nvidia-smi

Sets path to files

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

Install and import YOLOv8

In [ ]:
!pip install ultralytics==8.0.196

In [ ]:
from ultralytics import YOLO
from IPython.display import display, Image

# Get dataset

Dataset from roboflow

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="5s95xvyWyxq6A4qT3CrU")
project = rf.workspace("kakann").project("barbell-detector-bncfm")
dataset = project.version(2).download("yolov8")

# Train

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=100 imgsz=640 plots=True

# Display training results

In [ ]:
!pip install -q supervision==0.13.0
!pip install numpy

Display confusion matrix

In [ ]:
import supervision as sv
import numpy as np

ds = sv.DetectionDataset.from_yolo(
    images_directory_path=f"{dataset.location}/test/images",
    annotations_directory_path=f"{dataset.location}/test/labels",
    data_yaml_path=f"{dataset.location}/data.yaml",
)
model = YOLO('/content/best.pt')
def callback(image: np.ndarray) -> sv.Detections:
    result = model(image)[0]
    return sv.Detections.from_ultralytics(result)

confusion_matrix = sv.ConfusionMatrix.benchmark(
     dataset = ds,
     callback = callback
)
confusion_matrix_plot = confusion_matrix.plot()
confusion_matrix_plot.savefig(f"{HOME}/confusion_matrix.png")

Display results

In [ ]:
%cd {HOME}

results = Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)
display(results)

Display val batch labels and predictions

In [ ]:
%cd {HOME}

val_batch0_labels = Image(filename=f'{HOME}/runs/detect/train/val_batch0_labels.jpg', width=600)
val_batch0_pred = Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)
display(val_batch0_labels, val_batch0_pred)

In [ ]:
%cd {HOME}

val_batch1_labels = Image(filename=f'{HOME}/runs/detect/train/val_batch1_labels.jpg', width=600)
val_batch1_pred = Image(filename=f'{HOME}/runs/detect/train/val_batch1_pred.jpg', width=600)
display(val_batch1_labels, val_batch1_pred)

In [ ]:
%cd {HOME}

val_batch2_labels = Image(filename=f'{HOME}/runs/detect/train/val_batch2_labels.jpg', width=600)
val_batch2_pred = Image(filename=f'{HOME}/runs/detect/train/val_batch2_pred.jpg', width=600)
display(val_batch2_labels, val_batch2_pred)

Display curves

In [ ]:
%cd {HOME}

F1_curve = Image(filename=f'{HOME}/runs/detect/train/F1_curve.png', width=600)
PR_curve = Image(filename=f'{HOME}/runs/detect/train/PR_curve.png', width=600)
P_curve = Image(filename=f'{HOME}/runs/detect/train/P_curve.png', width=600)
R_curve = Image(filename=f'{HOME}/runs/detect/train/R_curve.png', width=600)
display(F1_curve, PR_curve, P_curve, R_curve)

Validation

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

# Download runs folder

Zip folder

In [ ]:
%cd {HOME}

!zip -r runs.zip {HOME}/runs

Download zip files

In [ ]:
from google.colab import files

files.download('runs.zip')